In [1]:
from __future__ import print_function
import os
import sys

In [2]:
# Someday this has to go in a config or kernel

spark_home = '/opt/spark/latest'

os.environ['SPARK_HOME'] = spark_home
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.1-src.zip'))

from pyspark.context import SparkContext
from pyspark.context import SparkConf
from pyspark.sql import SQLContext

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.load("data/idigbio-20161025.parquet")

In [5]:
print(df.count())
df.head(10)

72035127


[Row(barcodevalue=None, basisofrecord='preservedspecimen', canonicalname='russula albonigra (krombh.) fr., 1874', catalognumber='12758', class='agaricomycetes', collectioncode=None, collectionid='0cd2551b-8166-4c05-a0c9-5c8712ce0eb8', collector='k. a. harrison', commonname=None, continent='north america', country='canada', countrycode='can', datecollected=datetime.datetime(1972, 7, 14, 20, 0), family='russulaceae', fieldnumber=None, genus='russula', group=None, highertaxon=None, individualcount=None, infraspecificepithet=None, kingdom='fungi', municipality=None, order='russulales', phylum='basidiomycota', scientificname='russula albonigra', specificepithet='albonigra', stateprovince='nova scotia', uuid='21bf62fc-c3b7-4264-bf01-149bc9a36290', waterbody=None),
 Row(barcodevalue=None, basisofrecord='preservedspecimen', canonicalname='sorex haydeni baird, 1857', catalognumber='135621', class='mammalia', collectioncode='kum', collectionid=None, collector='kringen b', commonname=None, contin